In [1]:
import pandas as pd
import numpy as np

In [5]:
train_ = pd.read_csv('data/train_cases.csv')
train_.tail()

,caseid,age,sex,weight,height
3273,6381,55,F,69.50,169.8
3274,6385,69,M,62.30,159.3
3275,6386,61,F,43.25,151.7
3276,6387,24,F,55.50,155.7
3277,6388,47,F,51.50,150.0


In [8]:
del train_

In [4]:
vals = pd.read_csv('data/train_data/1.csv', header=None).values.flatten()

In [9]:
def moving_average(a, n=200):
    ret = np.nancumsum(a, dtype=np.float32)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [10]:
MINUTES_AHEAD = 5
SRATE = 100

##### 데이터가 너무 커서 , RAM이 32GB임에도 불구하고 부족함.
- 데이터를 4번에 걸쳐 나눠서 저장하는 식으로 진행

In [1]:
x_train = []
y_train = []
df_train = pd.read_csv('data/train_cases.csv')[:800]

from tqdm.notebook import tqdm



for _, row in tqdm(df_train.iterrows()):
    caseid = row['caseid']
    age = row['age']
    sex = row['sex']
    
    if sex=='M':
        row['sex'] = 1
    else:
        row['sex'] = 0
        
    weight = row['weight']
    height = row['height']
    
    
    
    bodyInfo = row[['age','sex','weight','height']].values
    
    vals = pd.read_csv('data/train_data/{}.csv'.format(caseid), header=None).values.flatten()

    # 20sec (20 00) - 5min (300 00) - 1min (60 00) = 38000 sample
    i = 0

    event_idx = []
    non_event_idx = []
    while i < len(vals) - SRATE * (20 + (1 + MINUTES_AHEAD) * 60):
        segx = vals[i:i + SRATE * 20]
        
        segy = vals[i + SRATE * (20 + MINUTES_AHEAD * 60):i + SRATE * (20 + (1 + MINUTES_AHEAD) * 60)]
            # 결측값 10% 이상이면
        if np.mean(np.isnan(segx)) > 0.1 or \
            np.mean(np.isnan(segy)) > 0.1 or \
            np.max(segx) > 200 or np.min(segx) < 20 or \
            np.max(segy) > 200 or np.min(segy) < 20 or \
            np.max(segx) - np.min(segx) < 30 or \
            np.max(segy) - np.min(segy) < 30 or \
            (np.abs(np.diff(segx[~np.isnan(segx)])) > 30).any() or \
            (np.abs(np.diff(segy[~np.isnan(segy)])) > 30).any():
            i += SRATE  # 1 sec 씩 전진
            continue
        segx = np.append(bodyInfo , segx) 

            # 출력 변수
        segy = moving_average(segy, 2 * SRATE)  # 2 sec moving avg
        event = 1 if np.nanmax(segy) < 65 else 0
        if event:  # event
            event_idx.append(i)
            x_train.append(segx)
            y_train.append(event)
        elif np.nanmin(segy) > 65:  # non event
            non_event_idx.append(i)
            x_train.append(segx)
            y_train.append(event)

        i += 30 * SRATE  # 30sec

    nsamp = len(event_idx) + len(non_event_idx)
    if nsamp > 0:
            print('{}: {} ({:.1f}%)'.format(caseid, nsamp, len(event_idx) * 100 / nsamp))

print('saving...', flush=True, end='')
x_train = np.array(x_train, dtype=np.float32)
y_train = np.array(y_train, dtype=bool)
np.savez_compressed('npy_bodyInfo/x_train.npz1', x_train)
np.savez_compressed('npy_bodyInfo/y_train.npz1', y_train)

print('done', flush=True)

In [5]:
print('1페이지 저장 완료 . . .')

1페이지 저장 완료 . . .


In [2]:
import numpy as np
x_train = []
y_train = []
df_train = pd.read_csv('data/train_cases.csv')[800:1600]

for _, row in tqdm(df_train.iterrows()):
    caseid = row['caseid']
    age = row['age']
    sex = row['sex']
    if sex=='M':
        row['sex'] = 1
    else:
        row['sex'] = 0
    weight = row['weight']
    height = row['height']
    bodyInfo = row[['age','sex','weight','height']].values

    
    vals = pd.read_csv('data/train_data/{}.csv'.format(caseid), header=None).values.flatten()

    # 20sec (20 00) - 5min (300 00) - 1min (60 00) = 38000 sample
    i = 0
    event_idx = []
    non_event_idx = []
    while i < len(vals) - SRATE * (20 + (1 + MINUTES_AHEAD) * 60):
        segx = vals[i:i + SRATE * 20]
        segy = vals[i + SRATE * (20 + MINUTES_AHEAD * 60):i + SRATE * (20 + (1 + MINUTES_AHEAD) * 60)]

            # 결측값 10% 이상이면
        if np.mean(np.isnan(segx)) > 0.1 or \
            np.mean(np.isnan(segy)) > 0.1 or \
            np.max(segx) > 200 or np.min(segx) < 20 or \
            np.max(segy) > 200 or np.min(segy) < 20 or \
            np.max(segx) - np.min(segx) < 30 or \
            np.max(segy) - np.min(segy) < 30 or \
            (np.abs(np.diff(segx[~np.isnan(segx)])) > 30).any() or \
            (np.abs(np.diff(segy[~np.isnan(segy)])) > 30).any():
            i += SRATE  # 1 sec 씩 전진
            continue
            
        segx = np.append(bodyInfo , segx) 
            # 출력 변수
        segy = moving_average(segy, 2 * SRATE)  # 2 sec moving avg
        event = 1 if np.nanmax(segy) < 65 else 0
        if event:  # event
            event_idx.append(i)
            x_train.append(segx)
            y_train.append(event)
        elif np.nanmin(segy) > 65:  # non event
            non_event_idx.append(i)
            x_train.append(segx)
            y_train.append(event)

        i += 30 * SRATE  # 30sec

    nsamp = len(event_idx) + len(non_event_idx)
    if nsamp > 0:
            print('{}: {} ({:.1f}%)'.format(caseid, nsamp, len(event_idx) * 100 / nsamp))

print('saving...', flush=True, end='')
x_train = np.array(x_train, dtype=np.float32)
y_train = np.array(y_train, dtype=bool)
np.savez_compressed('npy_bodyInfo/x_train.npz2', x_train)
np.savez_compressed('npy_bodyInfo/y_train.npz2', y_train)
print('done', flush=True)

In [ ]:
x_train = []
y_train = []
df_train = pd.read_csv('data/train_cases.csv')[2400:]

for _, row in tqdm(df_train.iterrows()):
    caseid = row['caseid']
    age = row['age']
    sex = row['sex']
    
    
    if sex=='M':
        row['sex'] = 1
    else:
        row['sex'] = 0
        
    weight = row['weight']
    height = row['height']
    bodyInfo = row[['age','sex','weight','height']].values
    
    vals = pd.read_csv('data/train_data/{}.csv'.format(caseid), header=None).values.flatten()

    # 20sec (20 00) - 5min (300 00) - 1min (60 00) = 38000 sample
    i = 0
    event_idx = []
    non_event_idx = []
    while i < len(vals) - SRATE * (20 + (1 + MINUTES_AHEAD) * 60):
        segx = vals[i:i + SRATE * 20]
        segy = vals[i + SRATE * (20 + MINUTES_AHEAD * 60):i + SRATE * (20 + (1 + MINUTES_AHEAD) * 60)]

            # 결측값 10% 이상이면
        if np.mean(np.isnan(segx)) > 0.1 or \
            np.mean(np.isnan(segy)) > 0.1 or \
            np.max(segx) > 200 or np.min(segx) < 20 or \
            np.max(segy) > 200 or np.min(segy) < 20 or \
            np.max(segx) - np.min(segx) < 30 or \
            np.max(segy) - np.min(segy) < 30 or \
            (np.abs(np.diff(segx[~np.isnan(segx)])) > 30).any() or \
            (np.abs(np.diff(segy[~np.isnan(segy)])) > 30).any():
            i += SRATE  # 1 sec 씩 전진
            continue
        segx = np.append(bodyInfo , segx) 
            # 출력 변수
        segy = moving_average(segy, 2 * SRATE)  # 2 sec moving avg
        event = 1 if np.nanmax(segy) < 65 else 0
        if event:  # event
            event_idx.append(i)
            x_train.append(segx)
            y_train.append(event)
        elif np.nanmin(segy) > 65:  # non event
            non_event_idx.append(i)
            x_train.append(segx)
            y_train.append(event)

        i += 30 * SRATE  # 30sec

    nsamp = len(event_idx) + len(non_event_idx)
    if nsamp > 0:
            print('{}: {} ({:.1f}%)'.format(caseid, nsamp, len(event_idx) * 100 / nsamp))

print('saving...', flush=True, end='')
x_train = np.array(x_train, dtype=np.float32)
y_train = np.array(y_train, dtype=bool)
np.savez_compressed('npy_bodyInfo/x_train.npz4', x_train)
np.savez_compressed('npy_bodyInfo/y_train.npz4', y_train)
print('done', flush=True)

In [ ]:
x_train = []
y_train = []
df_train = pd.read_csv('data/train_cases.csv')[1600:2400]

for _, row in tqdm(df_train.iterrows()):
    caseid = row['caseid']
    age = row['age']
    sex = row['sex']
          
    
    if sex=='M':
        row['sex'] = 1
    else:
        row['sex'] = 0
       
    weight = row['weight']
    height = row['height']
    bodyInfo = row[['age','sex','weight','height']].values

    vals = pd.read_csv('data/train_data/{}.csv'.format(caseid), header=None).values.flatten()

    # 20sec (20 00) - 5min (300 00) - 1min (60 00) = 38000 sample
    i = 0
    event_idx = []
    non_event_idx = []
    while i < len(vals) - SRATE * (20 + (1 + MINUTES_AHEAD) * 60):
        segx = vals[i:i + SRATE * 20]
        segy = vals[i + SRATE * (20 + MINUTES_AHEAD * 60):i + SRATE * (20 + (1 + MINUTES_AHEAD) * 60)]

            # 결측값 10% 이상이면
        if np.mean(np.isnan(segx)) > 0.1 or \
            np.mean(np.isnan(segy)) > 0.1 or \
            np.max(segx) > 200 or np.min(segx) < 20 or \
            np.max(segy) > 200 or np.min(segy) < 20 or \
            np.max(segx) - np.min(segx) < 30 or \
            np.max(segy) - np.min(segy) < 30 or \
            (np.abs(np.diff(segx[~np.isnan(segx)])) > 30).any() or \
            (np.abs(np.diff(segy[~np.isnan(segy)])) > 30).any():
            i += SRATE  # 1 sec 씩 전진
            continue
            
        segx = np.append(bodyInfo , segx) 
            # 출력 변수
        segy = moving_average(segy, 2 * SRATE)  # 2 sec moving avg
        event = 1 if np.nanmax(segy) < 65 else 0
        if event:  # event
            event_idx.append(i)
            x_train.append(segx)
            y_train.append(event)
        elif np.nanmin(segy) > 65:  # non event
            non_event_idx.append(i)
            x_train.append(segx)
            y_train.append(event)

        i += 30 * SRATE  # 30sec

    nsamp = len(event_idx) + len(non_event_idx)
    if nsamp > 0:
            print('{}: {} ({:.1f}%)'.format(caseid, nsamp, len(event_idx) * 100 / nsamp))

print('saving...', flush=True, end='')
x_train = np.array(x_train, dtype=np.float32)
y_train = np.array(y_train, dtype=bool)
np.savez_compressed('npy_bodyInfo/x_train.npz3', x_train)
np.savez_compressed('npy_bodyInfo/y_train.npz3', y_train)
print('done', flush=True)